# Monte Carlo to Estimate Stability of Feedback Systems
##### R. X Adhikari (2021)

Here what we want to do is:
* Make a plant and a controller
* Make some estimates of stability in the linear controller sense
* Run emcee to vary the plant parameters
* Use corner plots to visualize the posterior distribution of the stability metrics (? which are ???)


### Useful References
* [Feedback Controls by Astrom and Murray](http://www.cds.caltech.edu/~murray/amwiki/index.php/Second_Edition)
* [Why Null Measurements](https://courses.lumenlearning.com/physics/chapter/21-5-null-measurements/)
* [Gravitational Wave Detectors, Saulson (2017)](https://www.google.com/books/edition/_/5QVGDQEACAAJ?hl=en&sa=X&ved=2ahUKEwicmp_3v5TqAhUUJTQIHTdSDEoQre8FMAB6BAgAEA4)
* [Controls Tutorials](https://wiki.ligo.org/CSWG/Training_References) from the LIGO/Virgo/KAGRA Control Systems Working Group
* [State Space Methods](https://www.google.com/books/edition/Control_System_Design/2M1SAAAAMAAJ?hl=en)

In [ ]:
import sys,os
import numpy as np

from scipy import signal as sig
#from scipy import interpolate
from scipy.special import erfc # to integrate a Gaussian
#from scipy import optimize
import scipy.constants as const

# this is the Caltech py-controls package from Richard Murrary's group
# https://python-control.readthedocs.io/en/0.8.3/intro.html#installation
# Recommend installing in a dedicated Anaconda python environment:
# conda install -c conda-forge control
import control
import control.matlab as mat

# to check code execution times
from timeit import default_timer as timer

from IPython.display import Image, SVG, display

import matplotlib.pyplot as plt

import loopology
loopology.set_plot_params()

import warnings
from scipy.linalg import LinAlgWarning
warnings.filterwarnings("ignore", category=LinAlgWarning)

import emcee, corner
# uncomment if you have a Mac with Retina display
#%config InlineBackend.figure_format = 'retina'

# this is a nice jupyter lab theme:
# jupyter labextension install @oriolmirosa/jupyterlab_materialdarker



# params of the system
lambduh = 1064e-9  # [m] laser wavelength
w_0 = 2 *np.pi * const.c / lambduh
m_mirror = 40 # [kg] mirror mass



## Outline
1. Why Feedback?
1. Linear Systems
1. Laplace Domain
1. Feedback Loops
    1. Nomenclature
    1. How to calculate TFs for SISO
    1. MIMO calculations
        1. Adjacency Matrix
1. Noise Analysis
1. Range Analysis
1. Stability Analysis
1. Optimal Feedback

In [ ]:
SVG(filename='SISO-loop.svg')

# Example 1: Feedback Control of a Suspended Fabry-Perot Cavity
The Plant, P(s), includes the frequency response of the FP cavity:
$$ \frac{E_{\rm refl}}{x_{\rm mirror}}(\omega) = \frac{\rm DC~gain}{1 + i \omega/\omega_c}$$
as well as the pendulum motion of the mirror to an applied force:
$$ \frac{x(f)}{F(f)} = \frac{1/m}{\omega_0^2 + i \frac{\omega_0 \omega}{Q} - \omega^2}$$
where $\omega = 2 \pi f$ and $\omega_c$ is $2 \pi \times f_c$, the cavity pole frequency.

For this example, we 

In [ ]:
f = np.logspace(-1, 4, 500)
w = 2*np.pi * f

f_cav = 375 # cavity pole frequency [Hz]


# pendulum poles
f_pend = 1

def make_system(f_cav = 375, f_pend = 1, feedback_gain = 1):
    
    p1 = 1 * np.exp(1j * 88*np.pi/180)
    p_pend = -2*np.pi*np.array([p1, np.conj(p1)]) # must be complex conjugate
    
    
    # Convert the zpk model into state-space
    P = mat.zpk2ss([], p_pend, 1/m_mirror/(2*np.pi*f_pend)**2)
    Pend = control.ss(P[0], P[1], P[2], P[3])
    
    P = mat.zpk2ss([], [-2*np.pi*f_cav], 3e12)
    Cavity = control.ss(P[0], P[1], P[2], P[3])
    
    # the feedback controller
    C = mat.zpk2ss([-2*np.pi*30, -2*np.pi*30], [0, -2*np.pi*1500], 1e5 * feedback_gain)
    C = control.ss(C[0], C[1], C[2], C[3])
    
    # The Actuator, A, is flat up to 1111 Hz, with a constant force coeff [Newtons/Volt]
    A = mat.zpk2ss([],[-2*np.pi*1111],1)
    A = control.ss(A[0], A[1], A[2], A[3])
    
    # multiply them together 
    olg = control.series(Pend, Cavity, C, A)
    
    # unity gain feedback connection of the OLG to make CLG
    clg = control.feedback(1, olg)

    return olg, clg, Pend, Cavity, C, A


def process_system(olg, clg, Pend, Cavity, C, A, w):
    #w, mag, phase = sig.bode(olg, w=w)
    mag_pend, pha_pend, _ = control.freqresp(Pend,   omega=w)
    mag_cav,  pha_cav,  _ = control.freqresp(Cavity, omega=w)
    mag_act,  pha_act,  _ = control.freqresp(A,      omega=w)
    mag_c,    pha_c,    _ = control.freqresp(C,      omega=w)
    
    mag_olg, pha_olg, _ = control.freqresp(olg, omega=w)
    mag_clg, pha_clg, _ = control.freqresp(clg, omega=w)
    
    mag_clg  = mag_clg.flatten()
    mag_olg  = mag_olg.flatten()
    pha_clg  = pha_clg.flatten()
    pha_olg  = pha_olg.flatten()
    mag_pend = mag_pend.flatten()
    pha_pend = pha_pend.flatten()
    mag_cav  = mag_cav.flatten()
    pha_cav  = pha_cav.flatten()
    mag_c    = mag_c.flatten()
    pha_c    = pha_c.flatten()

    return mag_clg, mag_olg, pha_clg, pha_olg, mag_act, pha_act, mag_pend, pha_pend, mag_cav, pha_cav, mag_c, pha_c

# we can also calculate the RMS of e(t) and u(t) in the frequency domain, assuming that its a stationary process
def myrms(f, y_of_f):
    df = np.diff(f)
    df = np.append(df[0],df)
    #print(y_of_f.shape)
    y = df * (y_of_f)**2 #normalize the PSD
    y = np.cumsum(np.flipud(y)) # integrate the PSD from high frequency down to low freq to get the mean-squared
    rms = np.flipud(np.sqrt(y)) # flip it back to plot the RMS
    
    return rms.flatten()



In [ ]:
olg, clg, Pend, Cavity, C, A = make_system(f_cav = 375, f_pend = 1)
mag_clg, mag_olg, pha_clg, pha_olg, mag_act, pha_act, mag_pend, pha_pend, mag_cav, pha_cav, mag_c, pha_c = process_system(olg=olg, clg=clg, Pend=Pend, Cavity=Cavity, C=C, A=A, w=w)


#hclg = 1 / (1 - holg)

# plot the results
fig,ax = plt.subplots(2, 2, figsize=(19,10), sharex='col')

ax[0,0].semilogx(f, 20*np.log10(mag_pend * mag_cav), label='Plant', ls='--')
ax[0,0].semilogx(f, 20*np.log10(mag_c), label='Controller', ls='--')

ax[0,0].semilogx(f, 20*np.log10(mag_clg), label='CLG')
ax[0,0].semilogx(f, 20*np.log10(mag_olg), label='OLG')
ax[0,0].set_ylim([-60, 141])

ax[0,0].set_ylabel('Mag [dB]')
ax[0,0].set_xticklabels([])
ax[0,0].legend()


ax[1,0].semilogx(f, 180/np.pi*pha_pend, ls='--')
ax[1,0].semilogx(f, 180/np.pi*pha_c, ls='--')
ax[1,0].semilogx(f, 180/np.pi*pha_clg)
ax[1,0].semilogx(f, 180/np.pi*pha_olg)

ax[1,0].set_xlabel('Freq [Hz]')
ax[1,0].set_ylabel('Phase [deg]')
ax[1,0].set_ylim([-180, 181])
ax[1,0].set_yticks(np.arange(-180, 181, 60))


# now make the impulse and step response plots
T = np.linspace(0, 0.051, 1000)
t, y = control.impulse_response(clg,T)
ax[0,1].plot(t, y, label='Impulse Response')
ax[0,1].legend()

t, y = control.step_response(clg,T)
ax[1,1].plot(t, y, label='Step Response')
ax[1,1].set_xlabel('Time [s]')
ax[1,1].legend()


# Get the gain and phase margins from the Open Loop Gain
gm, pm, wg, wp = control.margin(mag_olg, 180/np.pi*pha_olg, w)
ax[0,0].axvline(wg/2/np.pi, lw=3, c='xkcd:Brown', alpha=0.3)
ax[1,0].axvline(wp/2/np.pi, lw=3, c='xkcd:Brown', alpha=0.3)
#gm, pm, wg, wp = control.margin(olg)

plt.show()
print('Kind of bogus Margins:')
print('Phase Margin = {:0.1f} deg.'.format(pm))
print('Gain Margin  = {:0.1f} dB.'.format(20*np.log10(gm)))
#K, CL, gam, rcond = control.hinfsyn(P, 1, 1)

# I don't think the infinite feedthrough or condition number warnings are that meaningful ?

## Noise Analysis
#### How to calculate the TF from the noise injection points to elsewhere in the system:
Simple - 
1. The loop suppresses the disturbance, $d$, so that right after the injection point: $$\textrm{in-loop noise} = d \times \frac{1}{1 - OLG}$$
1. From that point multiply by the forward loop gain between the injection point and the point of interest.

#### Example
* How big is the contribution of the force noise "Disturbance" in the error signal? After appling the CLG, we see that the only block between the injection of force noise and the error signal is the plant block, $P$:

$$\epsilon = d \times \frac{P}{1-OLG}$$ 

In the case of (quantum noise) for a single mirror in a Fabry-Perot cavity, we have:

$$F_{\rm RPN}(\omega) = \frac{2}{c} \sqrt{\frac{h c}{\lambda}\frac{P_{in}}{\omega_{cav}^2 + \omega^2}} $$ 

In [ ]:
# example radiation pressure noise
Pcav = 1e6 # [W] cavity power

F_RP = (2 / const.c) * np.sqrt(Pcav * const.hbar * w_0 / (1 + (w/(2*np.pi*f_cav))**2))
F_RP *= np.ones_like(w)

x_shot = 1e-20 * np.ones_like(w) * (1 + (w/(2*np.pi*f_cav)))

dissturb = (F_RP * mag_pend)**2 + (x_shot)**2
err = np.sqrt(dissturb)
err *= mag_clg

fig,ax = plt.subplots(1, figsize=(13,7))
ax.loglog(f, F_RP/m_mirror/w**2, label='RP noise')
ax.loglog(f, x_shot, label='shot noise')
ax.loglog(f, err, label='Closed-Loop Error Signal')

ax.set_ylim([1e-23, 1e-16])
ax.legend()
ax.set_xlabel(r'Frequency [Hz]')
ax.set_ylabel(r'Noise [m/$\sqrt{\rm Hz}$]')
ax.set_title('Sensitivity of the Interferometer')

plt.show()

## Optimal Feedback
There are many ways to design feedback loops. Classicaly, some of the non-algorithmic ones are:
* PID Control: standard textbook approaches - tune P first, then I, then D. All by looking at the step function response.
* Higher-order systems: inspection of Bode Plot and addition of poles & zeros "by hand"

The most simple form of optimal feedback is known as the Linear Quadratic Regulator (LQR). This method essentially minimizes a weighted combination of the feedback loop's error signal and control signal:

$$s = \int \epsilon(t)^2 + \alpha \int u(t)^2 $$
where $\epsilon(t)$ is the error signal, $u(t)$ is the feedback control signal, and $\alpha$ is a weight that the user adjusts to choose how much each term matters. For example, in some cases (e.g. a self driving car) it is important not only to keep the system well-controlled, but to also conserve energy (i.e. minimizing $u(t)^2$). 


In [ ]:


err_ASD = mag_pend * mag_cav * mag_clg * np.sqrt(dissturb)
err_RMS = myrms(f, err_ASD)

ctrl_ASD = err_ASD * mag_c * mag_act
ctrl_RMS = myrms(f, ctrl_ASD)

fig11, ax = plt.subplots(1, figsize=(16,12))
ax.loglog(f, F_RP/m_mirror/w**2, c='xkcd:Easter Purple', label='RP noise')
ax.loglog(f, x_shot, c='xkcd:Cement', label='shot noise')

ax.loglog(f, err_ASD, label='Closed-Loop Error Signal', c='xkcd:sky')
ax.loglog(f, err_RMS, c='xkcd:sky', ls='--', label='Error Signal (integrated RMS)')

ax.loglog(f, ctrl_ASD.flatten(), c='xkcd:Crimson', label='Control Signal')
ax.loglog(f, ctrl_RMS, c='xkcd:Crimson', ls='--', label='Control Signal (integrated RMS)')

ax.set_ylim([1e-23, 1e-16])
ax.legend()
ax.set_xlabel(r'Frequency [Hz]')
ax.set_ylabel(r'Noise [m/$\sqrt{\rm Hz}$]')
ax.set_title('Sensitivity of the Interferometer')

plt.show()

## Stability Analysis
There are many ways to analyze the loop stability:
1. Bode plot: check gain and phase margins as in OLG plot above
1. Lyapunov Stability
1. Root-Locus Plot
1. Nyquist Plot

In the ["Gang of 4"](http://robotics.caltech.edu/wiki/images/b/be/CDS110_Week9_Lecture1.pdf) plot below, we show a common 2x2 representation of the loop for making the stability analysis:
* $S = 1 / (1 + OLG)$; this is called the closed-loop gain, and also the "Sensitivity Function"
* $T = OLG / (1 + OLG)$; the "Complementary Sensitivity Function"
* $ P \times S$; the "Load Sensitivity Function"
* $ C \times S$; the "Noise Sensitivity Function"


## Range Analysis

We would like to ensure that the feedback signal we are using does not saturate the DAC nor the analog electronics. So let's compute the actuation range required for this.

We have already computed **Closed-Loop Error Signal** (above). So using our "loopology" algebra, we know how to compute the control signal spectrum:

$$ F_{ctrl}(\omega) = C(\omega) \times A(\omega) \times \epsilon(\omega)$$

So that's useful. That is the Amplitude Spectral Density ($ASD = \sqrt{PSD}$) of the **Force** applied to the mirror. To find out if we are saturating the actuator, we need to turn this into a time series quantity. We do this by integrating the [PSD](http://www.pmaweb.caltech.edu/Courses/ph136/yr2012/1206.1.K.pdf).

$$ F_{ctrl}^{RMS} = \sqrt{\int_{f_1}^{f_2} F_{ctrl}^2(\omega) d\omega}  $$

Well, that's swell. But so what? Well...now we assume that the noise processes, which the PSD represents, are non-white, [Gaussian noise](https://en.wikipedia.org/wiki/Gaussian_noise) sources. That means that amplitude of the noise, at any particular frequency, will have a Gaussian probability distribution around some mean. Since this is true for each frequency bin, we can assume (or prove it, if you like that kind of thing) that the sum of a bunch of Gaussians will also be a Gaussian. Actually, the Central Limit Theorem tells us that the sum of many, many whatever distributions will also be a Gaussian.

So we take a Gaussian with a Standard Deviation, $\sigma_F = F_{ctrl}^{RMS}$, and then ask:<br>
"For such a noise process, how often does the actuator try to exceed its maxium value?"<br>
and the answer to that is to just integrate the properly normalized Gaussian:

$$(\%~\mbox{of time we are saturating}) = 100 \times \int_{F_{max}}^{\infty} P(F) dF $$
which is just the complementary error function.

In [ ]:
fig,ax = plt.subplots(1,1)

fs = 16384 # sample rate of our digital controller

boo = np.linspace(0.01,5,1000)
yyy = erfc(boo) # complementary error function

ax.semilogy(boo, yyy)
ax.set_ylim([1e-12, 1])
ax.set_xlabel(r'$F_{max}$/$\sigma_F$')
ax.set_ylabel('Fraction of DAC samples which saturate')
ax.set_title('Actuator Saturations (f_sample = 16 kHz) assuming Gaussian noise')

ax.axhline(1/fs,       label='1 / second', c='xkcd:Red')
ax.axhline(1/fs/60,    label='1 / minute', c='xkcd:Orange')
ax.axhline(1/fs/60/60, label='1 / hour',   c='xkcd:Sea Green')
#ax.axhline(1/24/60/60)

ax.legend()
plt.show()
print(r'So, for a 16 kHz sample rate, we get a 5-$\sigma$ event approximately every {0:0.1f} days'.format(1/(fs*yyy[-1])/60/60/24))

### Definition of the Cost Function
However, its clear from the formula above that this method weights the overall RMS of the signal without regard to any frequency dependence. In the control systems world, there are then more sophisticated schemes to have different weighting, but in many cases where we analyze LTI systems it is more clear to produce a *Cost Function*, $s$, in the frequency domain.

Some considerations when choosing how to make an 'optimal' linear feedback controller:

1. Minimize a frequency-weighted version of the error signal: $\int W_{\epsilon}(f) \epsilon(f) df$
1. Minimize a frequency-weighted version of the control signal: $\int W_{ctrl}(f) u(f) df$
1. Minimize the rate of actuator saturations (i.e. minimize the *un-weighted* control signal)
1. Minimize the rate of sensors (ADC) saturations
1. avoid having any poles in the right half of the complex plane
1. Minimize the amount of gain peaking
1. Minimize the chance of loop instability for small, expected parameter variations.
1. Minimize the overall number of poles + zeros in the system (helps to reduce number of components in analog circuits or computation time in digital systems). It is not too helpful to shape the loop precisely for a very specific input noise spectrum unless you are sure that the spectrum will not change very much with time (true for shot noise, falso for seismic/acoustic noise).

In most practical cases, it does not make sense to make the overall cost function, $s$, depend quadratically on the cost. For example, we do not really care if we can reduce the rate of actuator saturations from 1/day to 1/month. Similarly, if the interferometer range is reduced to 1 Mpc, we do not really care if the range is further reduced to 0.5 Mpc - its all bad. So, when numerically seeking to optimize a multi-dimensional function like this, we want to make sure that the optimization manifold is well behaved, with no extremely huge mountains and no extremely deep valleys. This will allow the global optimization function to search efficiently.

### Variation of the Parameters
Once we're given the systems physical parameters and all of the noise inputs, the only thing we have left to design is the feedback controller, $C(s)$. We can think about this function in the following way:

$$ C(s) = \frac{num(s)}{den(s)}$$

where $num(s)$ and $den(s)$ are polynomials of order N. The roots of $num(s)$ are called the ***zeroes*** of the transfer function, and the roots of $den(s)$ are the ***poles***.
(you may have seen the use of poles and residues in classes on Complex Analysis or scattering problems in Quantum Mechanics, e.g. scattering of plane waves in 1D from any kind of potential well or barrier.)
To optimize the feedback system, we simply vary the values of the poles and zeros (or, equivalently, the coefficients of the polynomials) in the complex plane.

An alternative approach, called the partial fraction expansion seeks to make the optimization easier by representing $C(s)$ [as a sum of fractions](https://www.sintef.no/projectweb/vectorfitting/references/).

### Multi-Dimensional Search for optimum controller
Normal optimization methods based on stochastic gradient descent can often fail in multi-dimensional situations. The MCMC methods used for estimating the parameters of some experimental data can be used, by they are not as efficient at finding the optimum. They do however give more information about the shape of the search space.

Instead, we can use so-called Global Methods, such as Particle Swarm, Simulated Annealing, Genetic Algorithms, etc.

## Monte Carlo to estimate a stability cost
1. With the plant, P(s), and the controller, C(s), we close the loop.
1. Compute some values: low freq RMS, high freq RMS, etc.
1. Vary the plant parameters via Monte Carlo, and compute those values for each new plant
1. Histogram or Corner plot of results
1. How to quantify these outputs?

In [ ]:
# take as input some loop parameters
# calculate some scalars for the loop performance
# make a corner plot to show the results
def loop_perturb(f_cav = 375, f_pend = 1, feedback_gain = 1):
    
    # make the model
    olg, clg, Pend, Cavity, C, A = make_system(f_cav = f_cav, f_pend = f_pend, feedback_gain = feedback_gain)
    mag_clg, mag_olg, pha_clg, pha_olg, mag_act, pha_act, mag_pend, pha_pend, mag_cav, pha_cav, mag_c, pha_c = process_system(
                                                                                olg=olg, clg=clg, Pend=Pend, Cavity=Cavity, C=C, A=A, w=w)

    # compute the scalars: RMS, etc
    # now make the impulse resp
    T = np.linspace(0, 0.051, 1000)
    t_imp, y_imp = control.impulse_response(clg, T)
    imp_RMS = myrms(t_imp, y_imp)
    #ax[0,1].plot(t, y, label='Impulse Response')
    #ax[0,1].legend()

    # RMS of error point
    err_ASD = mag_pend * mag_cav * mag_clg * np.sqrt(dissturb)
    err_RMS = myrms(f, err_ASD)

    # RMS of control point
    ctrl_ASD = err_ASD * mag_c * mag_act
    ctrl_RMS = myrms(f, ctrl_ASD)
    
    return imp_RMS[0], err_RMS[0], ctrl_RMS[0]



# run a bunch of times with random plant values
N_costs   = 3 # how many metrics
N_params  = 3 # how many loop parameters
N_samples = 10000 # how many times to perturb the loop
samples   = np.zeros((N_samples, N_costs))
params    = np.zeros((N_samples, N_params))

# need to clip these values to avoid ridiculuous ness. eg. negative gains, etc
params[:,0] = np.random.normal(f_cav,  f_cav/10,  N_samples)
params[:,1] = np.random.normal(f_pend, f_pend/10, N_samples)
params[:,2] = np.random.normal(1,      1/10,      N_samples)


t_0 = timer()
for ix in range(N_samples):
    imp_RMS, err_RMS, ctrl_RMS = loop_perturb(f_cav = params[ix,0], f_pend = params[ix,1], feedback_gain = params[ix,2])
    samples[ix, 0] = imp_RMS
    samples[ix, 1] = err_RMS
    samples[ix, 2] = ctrl_RMS

t_elapsed = timer() - t_0
print('Elapsed time = {t:4.1f} seconds.'.format(t=t_elapsed))
# plot outputs in some clever histogram

In [ ]:
ll = 0.75
ul = 1.25
TV = [f_cav, f_pend, 1]
fig,ax = plt.subplots(nrows=1, ncols=3, figsize=(16,10))


colors = ['xkcd:Orange', 'xkcd:Green', 'xkcd:Burple']

for ii in range(N_costs):
    ax[ii].hist(samples[:,ii], bins = 40, density = False, 
             histtype ='bar', log=True,
             color = colors[ii],
             label = colors[ii])